In [35]:
import urllib2
import json
import pandas as pd

In [36]:
courses_response = urllib2.urlopen('https://api.coursera.org/api/catalog.v1/courses?fields=shortName,name,language&includes=universities,categories')
courses_data = json.load(courses_response)
courses_data = courses_data['elements']

In [37]:
courses_data[0]

{u'id': 69,
 u'language': u'en',
 u'links': {u'categories': [3, 8], u'universities': [10]},
 u'name': u'Contraception: Choices, Culture and Consequences',
 u'shortName': u'contraception'}

In [38]:
universities_response = urllib2.urlopen('https://api.coursera.org/api/catalog.v1/universities?fields=name,locationCountry')
universities_data = json.load(universities_response)
universities_data = universities_data['elements']

In [39]:
universities_data[0]

{u'id': 234,
 u'links': {},
 u'locationCountry': u'CN',
 u'name': u"Xi'an Jiaotong University",
 u'shortName': u'xjtu'}

In [40]:
categories_response = urllib2.urlopen('https://api.coursera.org/api/catalog.v1/categories')
categories_data = json.load(categories_response)
categories_data = categories_data['elements']

In [41]:
categories_data[0]

{u'id': 5, u'links': {}, u'name': u'Mathematics', u'shortName': u'math'}

In [42]:
courses_df = pd.DataFrame()

In [43]:
courses_df['course_name'] = map(lambda course_data: course_data['name'], courses_data)
courses_df['course_language'] = map(lambda course_data: course_data['language'], courses_data)
courses_df['course_short_name'] = map(lambda course_data: course_data['shortName'], courses_data)
courses_df['categories'] = map(lambda course_data: course_data['links']['categories'] if 'categories' in course_data['links'] else [], courses_data)
courses_df['universities'] = map(lambda course_data: course_data['links']['universities'] if 'universities' in course_data['links'] else [], courses_data)

In [44]:
courses_df.head()

,course_name,course_language,course_short_name,categories,universities
0,"Contraception: Choices, Culture and Consequences",en,contraception,"[3, 8]",[10]
1,Introduction to Computational Arts: Processing,en,compartsprocessing,"[1, 4, 18, 22]",[117]
2,Introduction to Computing 计算概论A,zh-cn,pkuic,"[1, 12]",[163]
3,Global Perspectives on Sexual and Reproductive...,en,globalsrhr,"[3, 8, 20]",[199]
4,Writing II: Rhetorical Composing,en,writing2,"[6, 14]",[28]


In [45]:
universities_df = pd.DataFrame()
universities_df['university_id'] = map(lambda university_data: university_data['id'], universities_data)
universities_df['university_name'] = map(lambda university_data: university_data['name'], universities_data)
universities_df['university_location_country'] = map(lambda university_data: university_data['locationCountry'], universities_data)

In [46]:
universities_df = universities_df.set_index('university_id')

In [47]:
universities_df.head()

,university_name,university_location_country
university_id,,
234,Xi'an Jiaotong University,CN
276,National Research Tomsk State University,RU
120,University of New Mexico,US
10,"University of California, San Francisco",US
56,"University of California, Santa Cruz",US


In [48]:
categories_df = pd.DataFrame()
categories_df['category_id'] = map(lambda category_data: category_data['id'], categories_data)
categories_df['category_name'] = map(lambda category_data: category_data['name'], categories_data)

In [49]:
categories_df = categories_df.set_index('category_id')

In [50]:
categories_df.head()

,category_name
category_id,
5,Mathematics
10,Biology & Life Sciences
24,Chemistry
25,Energy & Earth Sciences
14,Education


In [52]:
def map_ids_names(ids_array, df, object_name):
    names_array = []
    for object_id in ids_array:
        try:
            names_array.append(df.loc[object_id][object_name])
        except:
            continue
    return names_array

map_ids_names([4,5,15,16], categories_df, 'category_name')

[u'Information, Tech & Design',
 u'Mathematics',
 u'Engineering',
 u'Statistics and Data Analysis']

In [53]:
courses_df['categories_name'] = courses_df.apply(lambda row: map_ids_names(row['categories'], categories_df, 'category_name'), axis=1)
courses_df['universities_name'] = courses_df.apply(lambda row: map_ids_names(row['universities'], universities_df, 'university_name'), axis=1)

In [54]:
courses_df['course_url'] = 'https://www.coursera.org/course/' + courses_df['course_short_name']

In [55]:
courses_df.head()

,course_name,course_language,course_short_name,categories,universities,categories_name,universities_name,course_url
0,"Contraception: Choices, Culture and Consequences",en,contraception,"[3, 8]",[10],"[Medicine, Health & Society]","[University of California, San Francisco]",https://www.coursera.org/course/contraception
1,Introduction to Computational Arts: Processing,en,compartsprocessing,"[1, 4, 18, 22]",[117],"[Computer Science: Theory, Information, Tech &...",[The State University of New York],https://www.coursera.org/course/compartsproces...
2,Introduction to Computing 计算概论A,zh-cn,pkuic,"[1, 12]",[163],"[Computer Science: Theory, Computer Science: S...",[Peking University],https://www.coursera.org/course/pkuic
3,Global Perspectives on Sexual and Reproductive...,en,globalsrhr,"[3, 8, 20]",[199],"[Medicine, Health & Society, Social Sciences]",[Lund University],https://www.coursera.org/course/globalsrhr
4,Writing II: Rhetorical Composing,en,writing2,"[6, 14]",[28],"[Humanities , Education]",[The Ohio State University],https://www.coursera.org/course/writing2


In [56]:
SHAREDCOUNT_API_KEY = 'f061a8f426b0cc2ed50e3dcf7359594f13f46e8c'
def get_social_metrics(url, api_key):
    sharedcount_response = urllib2.urlopen('https://free.sharedcount.com/?url=' + url + '&apikey=' + api_key)
    return json.load(sharedcount_response)
courses_df['sharedcount_metrics'] = map(lambda course_url: get_social_metrics(course_url, SHAREDCOUNT_API_KEY), courses_df['course_url'])

In [57]:
courses_df['twitter_count'] = map(lambda sharedcount: sharedcount['Twitter'], courses_df['sharedcount_metrics'])
courses_df['linkedin_count'] = map(lambda sharedcount: sharedcount['LinkedIn'], courses_df['sharedcount_metrics'])
courses_df['facebook_count'] = map(lambda sharedcount: sharedcount['Facebook']['total_count'], courses_df['sharedcount_metrics'])

TypeError: 'int' object has no attribute '__getitem__'

In [58]:
courses_df.head()

,course_name,course_language,course_short_name,categories,universities,categories_name,universities_name,course_url,sharedcount_metrics,twitter_count,linkedin_count
0,"Contraception: Choices, Culture and Consequences",en,contraception,"[3, 8]",[10],"[Medicine, Health & Society]","[University of California, San Francisco]",https://www.coursera.org/course/contraception,"{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",0,3
1,Introduction to Computational Arts: Processing,en,compartsprocessing,"[1, 4, 18, 22]",[117],"[Computer Science: Theory, Information, Tech &...",[The State University of New York],https://www.coursera.org/course/compartsproces...,"{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",0,1
2,Introduction to Computing 计算概论A,zh-cn,pkuic,"[1, 12]",[163],"[Computer Science: Theory, Computer Science: S...",[Peking University],https://www.coursera.org/course/pkuic,"{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",0,0
3,Global Perspectives on Sexual and Reproductive...,en,globalsrhr,"[3, 8, 20]",[199],"[Medicine, Health & Society, Social Sciences]",[Lund University],https://www.coursera.org/course/globalsrhr,"{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",0,150
4,Writing II: Rhetorical Composing,en,writing2,"[6, 14]",[28],"[Humanities , Education]",[The Ohio State University],https://www.coursera.org/course/writing2,"{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",0,10


In [60]:
cols_to_show = ['course_name', 'universities_name', 'categories_name', 'twitter_count', 'linkedin_count']
#Get English courses
query = courses_df[courses_df['course_language'] == 'en']
#Sort the courses by twitter count and get the top 10 courses
query = query.sort('twitter_count', ascending=0).head(10)
query[cols_to_show]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,course_name,universities_name,categories_name,twitter_count,linkedin_count
0,"Contraception: Choices, Culture and Consequences","[University of California, San Francisco]","[Medicine, Health & Society]",0,3
262,Preparing for the AP* Calculus AB and BC Exams...,[University of Houston System],[Mathematics],0,0
287,Nudge-it: Understanding obesity,[The University of Edinburgh],"[Economics & Finance, Medicine, Health & Socie...",0,4
286,Teaching and Assessing Clinical Skills,[University of Michigan],"[Medicine, Health & Society]",0,9
284,Australian literature: a rough guide,[University of Western Australia],"[Humanities , Arts]",0,2
283,Building an Information Risk Management Toolkit,[University of Washington],"[Information, Tech & Design, Computer Science:...",0,69
282,Inquiry Science Learning: Perspectives & Pract...,[Rice University],"[Education, Teacher Professional Development]",0,0
280,Common Core in Action: Math Classroom Challeng...,[New Teacher Center],"[Mathematics, Education, Teacher Professional ...",0,4
279,"Living with Dementia: Impact on Individuals, C...",[Johns Hopkins University],"[Medicine, Health & Society, Social Sciences]",0,9914
277,"Networks: Friends, Money, and Bytes",[Princeton University],"[Economics & Finance, Information, Tech & Desi...",0,106


In [61]:
#Get English courses
query = courses_df[courses_df['course_language'] == 'en']
#Filter the "Statistics and Data Analysis" courses
query = query[query['categories_name'].map(lambda categories_name: 'Statistics and Data Analysis' in categories_name)]
#Sort the courses by twitter count and get the top 10 courses
query = query.sort('twitter_count', ascending=0).head(10)
query[cols_to_show]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,course_name,universities_name,categories_name,twitter_count,linkedin_count
30,Preparing for the AP* Statistics Exam,[University of Houston System],"[Mathematics, Statistics and Data Analysis]",0,0
237,Computational Molecular Evolution,[Technical University of Denmark (DTU)],"[Biology & Life Sciences, Statistics and Data ...",0,10
357,Startup Engineering,[Stanford University],"[Computer Science: Software Engineering, Busin...",0,254
341,High Performance Scientific Computing,[University of Washington],"[Information, Tech & Design, Statistics and Da...",0,63
338,Maps and the Geospatial Revolution,[The Pennsylvania State University],"[Information, Tech & Design, Statistics and Da...",0,327
315,Understanding the UK's 2015 General Election,[The University of Edinburgh],"[Humanities , Business & Management, Statistic...",0,3
295,AstroTech: The Science and Technology behind A...,[The University of Edinburgh],"[Physical & Earth Sciences, Statistics and Dat...",0,94
263,Applied Logistic Regression,[The Ohio State University],"[Economics & Finance, Medicine, Information, T...",0,70
259,The Caltech-JPL Summer School on Big Data Anal...,[Caltech],[Statistics and Data Analysis],0,9914
256,Enhance Your Career and Employability Skills,[University of London],"[Computer Science: Theory, Economics & Finance...",0,602
